In [1]:
#!/usr/bin/env python
# coding: utf-8
import os

from sklearn.model_selection import StratifiedKFold, train_test_split
from config import LSHS_DATAFILE, gpt_filtered_rephrase_lshs_file, HEVAL_DATAFILE
import numpy as np
import time
import pandas as pd
from tqdm import tqdm

In [2]:
import json
gpt_filtered_rephrase_tweets_file = gpt_filtered_rephrase_lshs_file

out_file = open(gpt_filtered_rephrase_tweets_file, "r")
filtered_rephrase_tweet_gpt = json.load(out_file)
out_file.close()

In [3]:
df = pd.read_csv(LSHS_DATAFILE)
domains = df['Domain'].unique().tolist()
for d in domains:
    print(d, df[df['Domain'] == d].shape)

Gender (9454, 4)
Religion (10869, 4)
Race (12013, 4)
Politics (11018, 4)
Sports (12306, 4)


In [4]:
# HateLabel	Final hate label decision 0-Normal, 1-Offensive, 2-Hate

In [5]:
# Convert to binary labels, combine Offensive and Hate as one class.
NUM_LABELS = 2
df['Label'] = df['Label'].replace(2, 1)


In [6]:
df

,Domain,TweetID,Tweet,Label
0,Gender,1344706773245038592,WATCH: Video previews #SurreyBC-shot film to f...,0
1,Gender,1344706877217792005,Men and women don’t have to solve their proble...,0
2,Gender,1344707261155962880,"At last I awake, very queer about the head, as...",0
3,Gender,1344707529213792256,WATCH: Video previews Surrey-shot film to focu...,0
4,Gender,1344709019865403394,heteronormativity is killing my people. how so...,1
...,...,...,...,...
55655,Sports,1277315350254751747,"Fuck off Gayle, professional footballer and yo...",1
55656,Sports,1277319456071581698,Omo I hate mancity abeg. What is this fluid fo...,1
55657,Sports,1277316487271854082,I hate playing Manchester United again,1
55658,Sports,1277319975305445381,I'll get trolled to fuck but I'd give anything...,1


In [7]:
# Labels:0 (nonhate) 1 (hate)
df_test = pd.read_csv(HEVAL_DATAFILE)
df_test = df_test.rename(columns={'text': 'Tweet', 'HS': 'Label', 'id': 'TweetID'})

In [8]:
df_test

,TweetID,Tweet,Label,TR,AG
0,34243,"@local1025 @njdotcom @GovMurphy Oh, I could ha...",0,0,0
1,30593,Several of the wild fires in #california and #...,0,0,0
2,31427,@JudicialWatch My question is how do you reset...,0,0,0
3,31694,"#Europe, you've got a problem! We must hurry...",1,0,0
4,31865,This is outrageous! #StopIllegalImmigration #...,1,0,0
...,...,...,...,...,...
2995,31368,you can never take a L off a real bitch😩 im ho...,1,1,0
2996,30104,@Brian_202 likes to call me a cunt & a bitch b...,1,1,0
2997,31912,@kusha1a @Camio_the_wise @shoe0nhead 1. Never ...,1,1,0
2998,31000,If i see and know you a hoe why would i hit yo...,1,1,0


# Setup for PyTorch-Lightning

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import DataLoader, Dataset
import torchmetrics
import torch
import pytorch_lightning as pl

/home/atif/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-23 19:01:05.956144: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-23 19:01:05.985173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 19:01:06.436281: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# https://wandb.ai/jack-morris/david-vs-goliath/reports/Does-Model-Size-Matter-A-Comparison-of-BERT-and-DistilBERT--VmlldzoxMDUxNzU
MAX_EPOCHS = 5 #5
BATCH_SIZE = 16*2 #+ int(55 * 0.9*0.5)
LEARNING_RATE = 1e-5
# MODEL_LLM = 'distilbert-base-uncased'
MODEL_LLM = 'bert-base-uncased'

# Setting the seed
pl.seed_everything(42, workers=True)

Global seed set to 42


42

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_LLM)
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [12]:
class MyDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    '''
    encoding.items() -> 
      -> input_ids : [1,34, 32, 67,...]
      -> attention_mask : [1,1,1,1,1,....]
    '''
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len((self.labels))

In [13]:
class LightningModel(pl.LightningModule):
    def __init__(self, model_name_or_path, num_labels, learning_rate=LEARNING_RATE):
        super().__init__()

        self.learning_rate = learning_rate
        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)

        # self.val_conf_mat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=NUM_LABELS)
        self.val_f1_macro_score = torchmetrics.classification.MulticlassF1Score(average="macro", num_classes=NUM_LABELS)
        self.val_f1_weighted_score = torchmetrics.classification.MulticlassF1Score(average="weighted", num_classes=NUM_LABELS)
        self.val_f1_non_avg_score = torchmetrics.classification.MulticlassF1Score(average="none", num_classes=NUM_LABELS)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_LABELS)
        
        # self.test_conf_mat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=NUM_LABELS)
        self.test_f1_macro_score = torchmetrics.classification.MulticlassF1Score(average="macro", num_classes=NUM_LABELS)
        self.test_f1_weighted_score = torchmetrics.classification.MulticlassF1Score(average="weighted", num_classes=NUM_LABELS)
        self.test_f1_non_avg_score = torchmetrics.classification.MulticlassF1Score(average="none", num_classes=NUM_LABELS)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_LABELS)
        
        self.metrics = {'val': [('val_f1_macro_score', self.val_f1_macro_score), ('val_f1_weighted_score', self.val_f1_weighted_score), ('val_acc', self.val_acc)],
                         'test': [('F1-Macro', self.test_f1_macro_score), ('F1-Weighted', self.test_f1_weighted_score),
                                  ('F1_Class 0', self.test_f1_non_avg_score.cpu()[0], 'test_f1_non_avg_score'), 
                                  ('F1_Class 1', self.test_f1_non_avg_score.cpu()[1], 'test_f1_non_avg_score'),
                                  ('Accuracy', self.test_acc)
                                 ]
                        }
        

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)
        
    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]  # this is passed to the optimizer for training

    def echo_metrics(self, key, predicted_labels, batch_labels):
        for itm in self.metrics[key]:
            if len(itm) == 2:
                mt_str, met = itm
                metric_attribute = None
            else:
                mt_str, met, metric_attribute = itm
                # print('this', mt_str, met, metric_attribute)
            met(predicted_labels, batch_labels)
            self.log(mt_str, met, prog_bar=True, metric_attribute=metric_attribute)
        
        
    
    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        self.log("val_loss", outputs["loss"], prog_bar=True)
        
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)

        self.echo_metrics('val', predicted_labels, batch["labels"])
        # print('#n here->',self.val_f1_non_avg_score(predicted_labels, batch["labels"]).cpu()[0])
        
        
    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.echo_metrics('test', predicted_labels, batch["labels"])

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Experiment

In [14]:
CF_LABEL = 0
import random
random.seed(42)

In [15]:
print(df['Label'].value_counts())
problematic_df = df[df['Label']>0]
problematic_df.shape

Label
0    44874
1    10786
Name: count, dtype: int64


(10786, 4)

In [16]:
def get_tweets():
    gpt_counterfactual_tweets = {}
    tot = problematic_df.shape[0]
    # print(tot)
    for i in range(0, tot):
        idx = problematic_df.iloc[i].name
        if str(i) in filtered_rephrase_tweet_gpt:
            gpt_counterfactual_tweets[idx] = filtered_rephrase_tweet_gpt[str(i)]
    return gpt_counterfactual_tweets
    
gpt_counterfactual_tweets = get_tweets()

In [17]:
import config
from nltk.tokenize import TweetTokenizer
import mosestokenizer
import numpy as np

def get_offensive_words():
    _df = pd.read_csv(config.en_swear_words_datafile, index_col=0)
    
    s = np.logical_or(_df['Level of offensiveness']=='Strongest words', _df['Level of offensiveness']=='Strong words')
    # display(_df[s]['Word'].to_list())
    wd_list = _df['Word'].to_list()
    
    _df = pd.read_csv(config.en_profanity_datafile, index_col=None)
    s = _df['severity_description'] == 'Severe'
    # wd_list.extend(_df[s]['text'].to_list())
    wd_list.extend(_df['text'].to_list())
    wd_list = set(map(str.lower, wd_list))
    return wd_list

offensive_wd_list = get_offensive_words()

In [18]:
def find_phrases(tokens, phrases):
    tokens = list(map(str.lower, tokens))
    """
    Find phrases in a list of sequential tokens.
    
    Args:
        tokens (list): List of sequential tokens.
        phrases (list): List of phrases to search for.
        
    Returns:
        A list of tuples containing the start and end index of each found phrase.
    """
    found_phrases = []
    
    for i in range(len(tokens)):
        for phrase in phrases:
            if tokens[i:i+len(phrase)] == phrase:
                found_phrases.append((i, i+len(phrase)-1))
    
    return found_phrases

def offensive_lexicon_used(t):
    tk = TweetTokenizer()
    detk = mosestokenizer.MosesDetokenizer('en')
    tk = tk.tokenize(t)
    # print(tk)
    phrase_index = find_phrases(tk, list(map(str.split, offensive_wd_list)))
    return len(phrase_index)

In [19]:
def get_counterfactual_tweets(data, labels, cf_label, single_cf_per_tweet=False, cf_size_prop_to_data=1.0, only_tweets_with_offensive_lexicon=True):
    tweets = []
    cnt =0 
    for idx in data.index:
        if idx in gpt_counterfactual_tweets:
            if (not only_tweets_with_offensive_lexicon) or offensive_lexicon_used(X[idx]):
                cnt += 1
                if not single_cf_per_tweet:
                    tweets.extend(gpt_counterfactual_tweets[idx])
                else:
                    tweets.append(gpt_counterfactual_tweets[idx][0])
    print('> Total Tweets used to generate counterfactuals ' + str(cnt))
    print('> Total counterfactuals added ' + str(len(tweets)))
    k = round(cf_size_prop_to_data * len(tweets))
    
    tweets = random.sample(tweets, k=k)
    print('> Counterfactual size ' + str(k) + ' at rate ' + str(cf_size_prop_to_data))
    cf_target = k*[cf_label]
    return pd.concat([data, pd.Series(tweets)], axis=0), pd.concat([labels, pd.Series(cf_target)], axis=0)

In [20]:
only_tweets_with_offensive_lexicon = False

In [21]:
domains

['Gender', 'Religion', 'Race', 'Politics', 'Sports']

In [22]:
# # only selected domains for this experiment
# domains = ['Gender', 'Race']

In [23]:
for d in domains:
    sel_df = df[df['Domain'] == d]
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)
    skf = StratifiedKFold(n_splits=2, random_state=None)
    for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
        print(splt_idx)
        X_train , X_test = X.iloc[train_index], X.iloc[test_index]
        y_train , y_test = y.iloc[train_index] , y.iloc[train_index]
        _,_ = get_counterfactual_tweets(
                X_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=.1, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)

Gender (9454, 4)
0
> Total Tweets used to generate counterfactuals 843
> Total counterfactuals added 843
> Counterfactual size 84 at rate 0.1
1
> Total Tweets used to generate counterfactuals 769
> Total counterfactuals added 769
> Counterfactual size 77 at rate 0.1
Religion (10869, 4)
0
> Total Tweets used to generate counterfactuals 766
> Total counterfactuals added 766
> Counterfactual size 77 at rate 0.1
1
> Total Tweets used to generate counterfactuals 767
> Total counterfactuals added 767
> Counterfactual size 77 at rate 0.1
Race (12013, 4)
0
> Total Tweets used to generate counterfactuals 564
> Total counterfactuals added 564
> Counterfactual size 56 at rate 0.1
1
> Total Tweets used to generate counterfactuals 526
> Total counterfactuals added 526
> Counterfactual size 53 at rate 0.1
Politics (11018, 4)
0
> Total Tweets used to generate counterfactuals 994
> Total counterfactuals added 994
> Counterfactual size 99 at rate 0.1
1
> Total Tweets used to generate counterfactuals 10

In [24]:
def __exp__(train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels, CF=False):
   
    if not CF:
        print('> Train samples', len(train_texts))
    else:
        print('> Train with CF samples', len(train_texts))
    
    train_encodings = tokenizer(list(train_texts), truncation = True, padding = True)
    valid_encodings = tokenizer(list(valid_texts), truncation = True, padding = True)
    test_encodings = tokenizer(list(test_texts), truncation = True, padding = True)

    #datasets
    train_dataset = MyDataset(train_encodings, train_labels)
    valid_dataset = MyDataset(valid_encodings, valid_labels)
    test_dataset = MyDataset(test_encodings, test_labels)
    
    #dataloaders
    bs = BATCH_SIZE
    train_loader = DataLoader(train_dataset, batch_size = bs, shuffle = True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size = bs, shuffle = True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size = bs, shuffle = True, num_workers=4)
    print(len(train_loader), len(valid_loader))

    # Setting the seed
    pl.seed_everything(42, workers=True)
    lightning_model = LightningModel(MODEL_LLM, NUM_LABELS)

    trainer = pl.Trainer(
        max_epochs=MAX_EPOCHS,
        accelerator="gpu",
        devices=1,
        deterministic=True,
        # log_every_n_steps=30,
        enable_checkpointing=True,  
        logger=False
    )
    
    trainer.fit(model=lightning_model,
                train_dataloaders=train_loader,
                val_dataloaders=valid_loader)
    
    r = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")
    del lightning_model
    del trainer
    return r



In [25]:
def get_splits(_X, _y, _X_test, _y_test, train_size=0.8):
    # 80-20 train-val size
    x_train, x_val, y_train, y_val = \
                    train_test_split(_X, _y, train_size=train_size)

    train_texts = x_train.values
    train_labels = y_train.values
    
    valid_texts = x_val.values
    valid_labels = y_val.values
    
    test_texts = _X_test.values
    test_labels = _y_test.values

    return x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels

def run_experiment_org(_X, _y, _X_test, _y_test):
    out_lst = []
    
    x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels = get_splits(_X, _y, _X_test, _y_test)
    org = __exp__(train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels, CF=False)

    l = [('splt_idx', -1, len(test_labels)),  ('train', len(train_labels)), {'Org': org}]
    out_lst.append(l)
    return out_lst

def run_experiment_counter_factuals(_X, _y, _X_test, _y_test, cf_size_prop_to_data=0.1):
    out_lst = []
    
    x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels = get_splits(_X, _y, _X_test, _y_test)
    x_train_with_cf, y_training_with_cf = get_counterfactual_tweets(
        x_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=cf_size_prop_to_data, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)
    train_texts_cf = x_train_with_cf.values
    train_labels_cf = y_training_with_cf.values

    cf = __exp__(train_texts_cf, train_labels_cf, valid_texts, valid_labels, test_texts, test_labels, CF=True)
    l = [('splt_idx', -1, len(test_labels)),  ('train', len(y_training_with_cf)), {'CF': cf}]
    out_lst.append(l)
    return out_lst

In [26]:
cf_size_prop_to_data_lst = list(np.arange(0.1, 1.1, 0.1))
try:
    del X, y
except:
    pass

In [ ]:
def run_now():
    complete_result = []
    out_dict = {}
    for d in domains:
        out_dict[d] = {}
        sel_df = df[df['Domain'] == d]
        print(d, sel_df.shape)
        X, y = sel_df['Tweet'], sel_df['Label'].astype(int)

        X_test, y_test = df_test['Tweet'], df_test['Label'].astype(int)
        res_bert_lst = run_experiment_org(X, y, X_test, y_test)
        out_dict[d]['Org'] = res_bert_lst
        out_dict[d]['CF'] = {}
        for cf_size_prop_to_data in cf_size_prop_to_data_lst:
            res_bert_lst = run_experiment_counter_factuals(X, y, X_test, y_test, cf_size_prop_to_data=cf_size_prop_to_data)
            out_dict[d]['CF'][cf_size_prop_to_data] = res_bert_lst
        complete_result.append(out_dict)
        json.dump(out_dict, open('out/'+ MODEL_LLM + '-EP_'+ str(MAX_EPOCHS) + '-ft-lshd22-heval19-ood.json', 'w'))
    return complete_result

start = time.time()
complete_result = run_now()
end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

Gender (9454, 4)
> Train samples 7563


Global seed set to 42


237 60


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Sanity Checking: 0it [00:00, ?it/s]

...

284 68


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Sanity Checking: 0it [00:00, ?it/s]

/home/atif/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Epoch 0: 100%|████████████████████████████████| 284/284 [00:53<00:00,  5.30it/s]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  81%|██████████████▌   | 55/68 [00:03<00:00, 17.92it/s]

In [28]:
def run_now_full():
    complete_result = []
    out_dict = {}
    d = 'Complete'
    out_dict[d] = {}
    sel_df = df
    print(d, sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)

    X_test, y_test = df_test['Tweet'], df_test['Label'].astype(int)
    res_bert_lst = run_experiment_org(X, y, X_test, y_test)
    out_dict[d]['Org'] = res_bert_lst
    out_dict[d]['CF'] = {}
    for cf_size_prop_to_data in cf_size_prop_to_data_lst:
        res_bert_lst = run_experiment_counter_factuals(X, y, X_test, y_test, cf_size_prop_to_data=cf_size_prop_to_data)
        out_dict[d]['CF'][cf_size_prop_to_data] = res_bert_lst
    complete_result.append(out_dict)
    json.dump(out_dict, open('out/'+ MODEL_LLM + '-EP_'+ str(MAX_EPOCHS) + '-ft-lshd22-heval19-ood-full.json', 'w'))
    return complete_result


start = time.time()
complete_result = run_now_full()
end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

Complete (55660, 4)
> Train samples 44528


Global seed set to 42


1392 348


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Sanity Checking: 0it [00:00, ?it/s]

...

Epoch 0: 100%|██████████████████████████████| 1597/1597 [08:33<00:00,  3.11it/s]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|███████████████▉| 347/348 [00:25<00:00, 13.66it/s]
Epoch 0: 100%|█| 1597/1597 [08:59<00:00,  2.96it/s, val_loss=0.127, val_f1_macro
Epoch 1: 100%|█| 1597/1597 [08:33<00:00,  3.11it/s, val_loss=0.127, val_f1_macro
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|███████████████▉| 347/348 [00:25<00:00, 13.64it/s]
Epoch 1: 100%|█| 1597/1597 [08:59<00:00,  2.96it/s, val_loss=0.124, val_f1_macro
Epoch 2: 100%|█| 1597/1597 [08:34<00:00,  3.11it/s, val_loss=0.124, val_f1_macro
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|███████████████▉| 347/348 [00:25<00:00, 13.64it/s]
Epoch 2: 100%|█| 1597/1597 [09:00<00:00,  2.96it/s, val_loss=0.147, val_f1_macro
Epoch 3: 100%|█| 1597/1597 [08:34<00:00,  3.11it/s, val_loss=0.147, val_f1_macro
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0: 100%|███████████████▉| 347/348 [00:25<00:

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 1597/1597 [09:01<00:00,  2.95it/s, val_loss=0.203, val_f1_macro


You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /home/atif/work/notebook-data/chatgpt/checkpoints/epoch=4-step=7985.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/atif/work/notebook-data/chatgpt/checkpoints/epoch=4-step=7985.ckpt
/home/atif/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing DataLoader 0: 100%|█████████████████████| 94/94 [00:13<00:00,  7.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Accuracy            0.5356666445732117
        F1-Macro            0.5354746580123901
       F1-Weighted          0.5369858741760254
       F1_Class 0           0.5449199676513672
       F1_Class 1           0.5260292887687683
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Time elapsed 371.38 min
